<a href="https://colab.research.google.com/github/thofes/judging_SJ_BA/blob/main/Mark_11_yolo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#Connect GDrive
from google.colab import drive
drive.mount('/content.gdrive')

#init Repository - YOLOv5
%cd /content


!git clone https://github.com/thofes/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt  # install

Mounted at /content.gdrive
/content
Cloning into 'yolov5'...
remote: Enumerating objects: 11163, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 11163 (delta 0), reused 2 (delta 0), pack-reused 11160
Receiving objects: 100% (11163/11163), 11.19 MiB | 15.03 MiB/s, done.
Resolving deltas: 100% (7700/7700), done.
/content/yolov5
     |████████████████████████████████| 596 kB 17.2 MB/s 


Exception falls gar nichts entdeckt wir dim BIld

In [ ]:
import os
from os.path import exists, join, basename, splitext
import math
from collections import OrderedDict
import json
import numpy as np
import cv2


PATH = "/content.gdrive/My Drive/Try"
label= []
try:
    label = next(os.walk(PATH))[1]
except StopIteration:
    pass
#data["labels"].append(label)
print(label)

for l in label:
  Path_jumpers = PATH + '/' + l
  print(Path_jumpers)
  Jumpers = []
  try:
    Jumpers = next(os.walk(Path_jumpers))[1]
  except StopIteration:
    pass
  print(Jumpers)

  for j in Jumpers:
    Path_Pic = Path_jumpers + '/' + j
    print(Path_Pic)

    %cd /content/yolov5
    !python detect.py --save-crop --source '{Path_Pic}' --project '{Path_Pic}' --save-txt


    with open(Path_Pic + '/exp/predictions.json', "r") as fp:
        data = json.load(fp)

    # convert lists to numpy arrays
    x_center = np.array(data["x_center"])
    y_center = np.array(data["y_center"])
    x1 = np.array(data["x1"])
    x2 = np.array(data["x2"])
    y1 = np.array(data["y1"])
    y2 = np.array(data["y2"])
    det = np.array(data["det"])
    width = 1920
    hight = 1080




    #detect movement mit check vor und zurück


    athlet_det_index = []
    y_centers = []

    for y in range(21):
      #if det(y) != 0: wird eventuellenötigt, falls keine detections gemacht wurden
        distance_next = []
        distance_index_next = []
        distance_previous = []
        distance_index_previous = []
        valid_dist_index_next = []
        valid_dist_index_previous = []
        valid_dist_index = []
        y_center_per_det = []
        y_center_abs_per_det = []

        if y == 0: #beim ersten Bild wird der Abstand der BB zum Folgebild berrechnet
            
            for x in range(det[y+1]):
                distance_index_previous.append(x) # wird aufgefüllt damit beim Vergleich nur die Indexe der distance_next Liste vorhanden bleiben            
                for i in range(det[y]):
                    distance_next.append(math.sqrt((x_center[sum(det[:y+1])+x]-x_center[sum(det[:y])+i])**2 + (y_center[sum(det[:y+1])+x]-y_center[sum(det[:y])+i])**2))
                    distance_index_next.append(i)

        if y > 0 and y < 20: #bei den mittleren Bildern wird der Abstand der BB zum Vorbild und Folgebild berrechnet
            
            for x in range(det[y+1]):
                
                for i in range(det[y]):
                    distance_next.append(math.sqrt((x_center[sum(det[:y+1])+x]-x_center[sum(det[:y])+i])**2 + (y_center[sum(det[:y+1])+x]-y_center[sum(det[:y])+i])**2))
                    distance_index_next.append(i)

            for x in range(det[y-1]):
                
                for i in range(det[y]):
                    distance_previous.append(math.sqrt((x_center[sum(det[:y-1])+x]-x_center[sum(det[:y])+i])**2 + (y_center[sum(det[:y-1])+x]-y_center[sum(det[:y])+i])**2))
                    distance_index_previous.append(i)
            #print(distance)
            
                  
        else: # Beim letzten Bild wird der Abstand zur BB des vorletzten Bildes berechnet
            for x in range(det[y-1]):
                
                for i in range(det[y]):
                    distance_previous.append(math.sqrt((x_center[sum(det[:y-1])+x]-x_center[sum(det[:y])+i])**2 + (y_center[sum(det[:y-1])+x]-y_center[sum(det[:y])+i])**2))
                    distance_index_previous.append(i)
                  
            #print("dist: ", distance)
            #print(distance_index)

        for x in range(det[y]): # y Koordinaten des Zentrums in Liste speichern
            y_center_per_det.append(y_center[sum(det[:y])+x])
            

        y_centers.append(y_center_per_det) # Erstellung einer Liste mit den Listen der einzelnen y Koordinaten der Detections in jedem Bild

        if y == 0: # beim ersten Bild nur vergleich mit dem nächsten
            for i, x in enumerate(distance_next): # Einschränkung der Detections anhand des Abstandes zum nächsten Bild
                if x > 15 and x < 70:
                    valid_dist_index.append(distance_index_next[i])

        if y > 0 and y < 20: # bei den mittleren Bildern werden beide Richtungen eingeschränkt
            for i, x in enumerate(distance_next): # Einschränkung der Detections anhand des Abstandes zum nächsten Bild
                if x > 15 and x < 70:
                    valid_dist_index_next.append(distance_index_next[i])

            for i, x in enumerate(distance_previous): # Einschränkung der Detections anhand des Abstandes zum vorherigen Bild
                if x > 15 and x < 70:
                    valid_dist_index_previous.append(distance_index_previous[i])

            print("next: ",valid_dist_index_next)
            print("prev: ",valid_dist_index_previous)
            # Listeneinträge vergleichen und Übereinstimmungen zusammenführen
            s1 = set(valid_dist_index_next)
            s2 = set(valid_dist_index_previous)
            s3 = s1.intersection(s2)
            valid_dist_index = list(s3)

        if y == 20: # beim letzten Bid wieder nur vergleich mit dem vorherigen Bild
            for i, x in enumerate(distance_previous): # Einschränkung der Detections anhand des Abstandes zum vorherigen Bild
                if x > 15 and x < 70:
                    valid_dist_index.append(distance_index_previous[i])
        




        print("List: ", valid_dist_index)

        if len(valid_dist_index) > 1: # Falls für eine Bildbox mehrere mögliche Verbindungen zum Folgebild gefunden wurden, werden die Duplikate entfernt
            valid_dist_index = list(OrderedDict.fromkeys(valid_dist_index))

        if len(valid_dist_index) == 1: # wenn nur eine vorhanden ist wird diese verwendet
            athlet_det_index.append(valid_dist_index[0])

        if y == 0: # Für das erste Bild
            
            if len(valid_dist_index) > 1: # falls mehrere möglich sind, wird die BB mit der zentralste Y Koordinate gewählt
                for dist in valid_dist_index:
                    y_center_abs_per_det.append(abs(y_center[sum(det[:y])+dist]-hight/2))
                closest = min(y_center_abs_per_det)
                min_index = y_center_abs_per_det.index(closest)            
                athlet_det_index.append(valid_dist_index[min_index]) # index der Detection vom ersten Bild gespeichert
            
            elif len(valid_dist_index) == 0: # falls keine Movements aus den BB erkannt werden - auch zetralste Y-Koordinate
                for x in range(det[y]):
                    y_center_abs_per_det.append(abs(y_center[sum(det[:y])+x]-hight/2)) 
                closest = min(y_center_abs_per_det)
                min_index = y_center_abs_per_det.index(closest)
                athlet_det_index.append(min_index) # index der Detection vom ersten Bild gespeichert


        elif y < 21: # Für alle weiteren Bilder 

            if len(valid_dist_index) > 1: # falls mehrere möglich sind, wird jene BB mit dem geringsten Abstand zur Y-Koordinate der vorher geählten BB ausgewählt
                dist_to_previous_y_center = []

                for i, x in enumerate(valid_dist_index):
                    dist_to_previous_y_center.append(abs(y_center_per_det[x]-y_center[sum(det[:y-1])+athlet_det_index[y-1]]))
                closest = min(dist_to_previous_y_center)
                min_index = dist_to_previous_y_center.index(closest)
                athlet_det_index.append(valid_dist_index[min_index]) # index der Detection wird gespeichert           

                #optional einfach die zentralste Detection laut Y-Kooridnate
                #for x in range(det[y]):
                    #y_center_abs_per_det.append(abs(y_center[sum(det[:y])+x]-hight/2)) 
                #closest = min(y_center_abs_per_det)
                #min_index = y_center_abs_per_det.index(closest)
                #athlet_det_index.append(valid_dist_index[min_index]) # index der Detection vom Bild wird gespeichert

            if len(valid_dist_index) == 0: #falls keine Movements aus den BB erkannt werden - auch zetralste Y-Koordinate oder vergleichen mit dem vorherigen Bild
                print("BP")


    print(athlet_det_index)










    #crop images
    for i,x in enumerate(athlet_det_index):
        if x != 'X':
            #print(sum(det[:i]))
            #print(x)
            x_1 = x1[sum(det[:i])+x]-50
            x_2 = x2[sum(det[:i])+x]+50
            y_1 = y1[sum(det[:i])+x]-50
            y_2 = y2[sum(det[:i])+x]+50

            if x_1 < 0:
                x_1 = 0

            if x_2 > width:
                x_2 = width

            if y_1 < 0:
                y_1 = 0

            if y_2 > hight:
                y_2 = hight


            path = Path_Pic + '/' + str(i+1) + '.jpg'
            print(path)
            im = cv2.imread(path)
            #print(x_1, x_2, y_1, y_2)
            crop = im[int(y_1):int(y_2), int(x_1):int(x_2)] 
            #save_dir.parent.mkdir(parents=True, exist_ok=True)  # make directory
            save_path = Path_Pic + '/exp/' + str(i+1) + '.jpg'
            cv2.imwrite(save_path, crop) # speicher nur den zugeschnittenen Springer





